# name-that-building

In [0]:
!wget https://github.com/rohank07/ML_A3/blob/master/model.zip?raw=true
!unzip model.zip?raw=true

from keras.models import load_model
from google.colab import files


In [0]:
import numpy as np
from PIL import Image
import matplotlib.pyplot as plt
from keras.models import load_model
# List of building codes to use throughout notebook.
buildingCodes = ('AA', 'CB', 'CT', 'DT', 'FH', 'HP', 'HS', 'LB', 'MC', 'ME', 'ML', 'PA', 'RB', 'RO', 'SA', 'TB')

def prepareModel():
  global model # Sample global variable that you may populate here.
  model = load_model('model/model.h5')


In [0]:
def classifyImage(fname):
  global model
  tdf = pd.DataFrame(columns=["Image","label"])
  tdf.loc[0] = [fname,""]
  pred_gen = ImageDataGenerator(rescale = 1./255,
    fill_mode='nearest',
    width_shift_range=0.2,
    height_shift_range=0.2)
  predict_generator = pred_gen.flow_from_dataframe(
      tdf,
      x_col= "x_col", 
      y_col = "y_col", 
      target_size=(500,500), 
      batch_size=50)
  print("Predicting class of '{0:s}' using model '{1:s}'".format(fname,model))
  predict = model.predict(predict_generator)
  return buildingCodes[np.argmax(predict[0])] 

In [0]:
def worstRecall():
  return buildingCodes[12] #Default behaviour to override


In [0]:
def bestPrecision():
  return buildingCodes[3] #Default behaviour to override


In [0]:
# Function to return estimated accuracy that will be obtained across all test images
def estimatedAccuracy():
  return 0.07258065 

In [0]:
# Function to return estimated accuracy that will be obtained across all test images
def estimatedAccuracy():
  return 1.0 #Default behaviour to override

In [0]:
import numpy as np
from PIL import Image, ExifTags
import matplotlib.pyplot as plt
from glob import glob


# First prepare the model:
prepareModel() # Limited to 5 minutes...

# Load (secret) test data into local Colab environment
!wget https://github.com/jrgreen7/SYSC4906/blob/master/Assignments/Assignment3/Images/SampleImages.zip?raw=true
!unzip SampleImages.zip?raw=true #Should create 4 images
test_images = sorted(glob('SampleImages/*.jpg'))
actual_labels = (buildingCodes[0],buildingCodes[1],buildingCodes[3],buildingCodes[6])

# Classify sample test images:
TP = 0
for imgFname,actual_label in (zip(test_images,actual_labels)):
  pred_label = classifyImage(imgFname) # Predict the label of this image file

  # Plot the image with actual and predicted labels
  # Note that we may have to rotate the image, depending on the 
  # orientation of the camera. Use EXIF tags for this:
  im = Image.open(imgFname)
  for orientation in ExifTags.TAGS.keys() : 
    if ExifTags.TAGS[orientation]=='Orientation' : break 
  exif=dict(im._getexif().items())

  if exif[orientation] == 3 : 
    im=im.rotate(180, expand=True)
  elif exif[orientation] == 6 : 
    im=im.rotate(270, expand=True)
  elif exif[orientation] == 8 : 
    im=im.rotate(90, expand=True)

  im = im.convert('RGB') # May not be necessary?

  plt.title("Sample test image of {0:s} predicted as {1:s}".format(actual_label, pred_label))
  plt.imshow(np.asarray(im))
  plt.axis('off')
  plt.show()

  if (pred_label==actual_label):
    print('Correct!')
    TP += 1
  else:
    print("Incorrect...")

# Print the predicted performance:
print("Expected that worst recall would be on {0:s}".format(worstRecall()))
print("Expected that best precision would be on {0:s}".format(bestPrecision()))
print("Expected total accuracy would be {0:.3f}".format(estimatedAccuracy()))
print("Actual total accuracy is {0:.3f}".format((TP)/(len(test_images))))
